In [2]:
import sys
sys.path
sys.path.append('../')

import numpy as np 
import pandas as pd

from data import RandomData, AmazonBooks, ToyData, MovieLensData
from model import SimpleMeanModel, UserMeanModel, ProductMeanModel, CombinedMeanModel

%matplotlib inline

In [3]:
%load_ext autoreload
%autoreload 2

### Load Data

In [4]:
# ds = MovieLensData(min_user_ratings=5).get_dataset(verbose=True)
ds = AmazonBooks(min_user_ratings=10).get_dataset(verbose=True)
train = ds['train']
val = ds['val']

loading preprocessed dataset from disk


In [40]:
print(train.keys())
train['product_reviews'].head()

dict_keys(['user_product_ratings', 'product_descriptions', 'product_reviews'])


,product_id,review
0,6155,for getting your kid introduced to his/her ABC...
1,6155,"This Book is funny and is full of B words, lik..."
2,6155,A favorite Berenstain book of my children I wa...
3,6155,This book is quite funny. Especially when you...
4,6155,Teaching the next generation to love books! M...


## create matrix: product_id -> all text about that product

all text includes all reviews about it and the description of that product

In [115]:
# grouped_reviews = train['product_reviews'].groupby(['product_id']).get_group(6155)
# grouped_reviews['review'].agg([np.concatenate])
grouped_reviews = train['product_reviews'].groupby('product_id')['product_id', 'review'].aggregate(\
        {'product_id':['mean'], 'review':lambda x: list(x)})
grouped_reviews .columns = ['product_id', 'review']

In [116]:
grouped_reviews.head()

,product_id,review
product_id,,
2,2,"[When I lost my best furry friend of 16 years,..."
3,3,[I'm reading this for the second time. The bo...
4,4,[This book had an excellent theme: ugly duckli...
5,5,[I have always enjoyed romance novels and Donn...
6,6,"[This is the second level of ""Speed Secrets"". ..."


In [250]:
# print(list(grouped_reviews.loc[grouped_reviews['product_id'] == 6155]['review'])[0])

In [320]:
combined = train['product_descriptions'].merge(grouped_reviews, on='product_id')
combined['all_text'] = combined['review']
combined['all_text'] = combined['description'].apply(lambda x: [x]) +  combined['all_text']
combined.head()

,description,product_id,review,all_text
0,"""Fairy phobic or trapped in a true-life fairy ...",12572,[Dream come true or worst nightmare? Grace Mac...,"[""Fairy phobic or trapped in a true-life fairy..."
1,"Welcome back, Mr Bond. We've been waiting for...",4397,"[Okay 007 fans, this book was a waste of time....","[Welcome back, Mr Bond. We've been waiting fo..."
2,"""A heart-wrenching and uplifting story of surv...",4175,"[Amazing book!It is very detailed, telling the...","[""A heart-wrenching and uplifting story of sur..."
3,A fast-paced and engrossing story. A definite ...,9448,[Lori-Bryant Woolridge is one of my favorite a...,[A fast-paced and engrossing story. A definite...
4,Gayle Jackson Sloan is a native of Philadelphi...,14535,[Wednesday's Woes was an excellent piece of ar...,[Gayle Jackson Sloan is a native of Philadelph...


In [221]:
# print(list(combined.loc[combined['product_id'] == 6155]['review'])[0])
# print()
# print(" EOF ".join(list(combined.loc[combined['product_id'] == 6155]['all_text'])[0])
ls = combined.loc[combined['product_id'] == 12572]['all_text']
print(ls.tolist()[0])

['"Fairy phobic or trapped in a true-life fairy adventure? Grace\'s search for help lands her in the office..."  -- Author Cheryl St.John "Fuhrmann\'s three hilarious fairies are superb! Their good intentions bring about delightful situations you will love."  -- Shari Brennan, Simegen.com"This romantic comedy has all the elements--engaging characters, an adorable tale, and scenes that make you laugh out loud."  -- Laura Baker, award-winning author, Legend\tBook #1 in the Fairy Godmother series\tHolly Fuhrmann lives in Erie, PA and lives with one husband, four children, and one drooling Old English Mastiff, which makes her a leading authority on sanity-challenged writers.', "Dream come true or worst nightmare? Grace Macguire can't figure out what in the world is going on, when three of the characters she'd created for recurring roles in her romance novels suddenly seem to come to life and decided that *she* is their next 'Cinderella Story.'On the pretense of doing research, Grace enlist

## create matrix: product_id , vocab matrix

Rows correspond to product id's, columns to 5000 most common words in vocab

In [233]:
from collections import Counter
def get_vocab(X, n_words=None):
    """Get the vocabulary for an RNN example matrix `X`,
    adding $UNK$ if it isn't already present.

    Parameters
    ----------
    X : list of lists of str
    n_words : int or None
        If this is `int > 0`, keep only the top `n_words` by frequency.

    Returns
    -------
    list of str

    """
    wc = Counter([w for ex in X for w in ex])
    wc = wc.most_common(n_words) if n_words else wc.items()
    vocab = {w for w, c in wc}
    vocab.add("$UNK")
    return sorted(vocab)

In [378]:
combined['all_text_parsed'] = combined['all_text'].apply(lambda x: re.sub("[,.;?!():\[\]]",""," ".join(x)).lower().split(" "))
# first join all comments reviews, etcetera into 1 big string
# then remove all punctuation
# then split on spaces to create words

In [377]:
import re
re.sub("[,.;?!():\[\]]","" , "a[df,e(ar;af.a)s]df!a:c?")

'adfearafasdfac'

In [383]:
vocab = get_vocab(combined['all_text_parsed'].tolist(), 30000)

In [382]:
# TODO: remove punctuation
print(vocab)

157571


In [384]:
combined_words = combined[['product_id','all_text_parsed']]
combined_words.head()

,product_id,all_text_parsed
0,12572,"[""fairy, phobic, or, trapped, in, a, true-life..."
1,4397,"[welcome, back, mr, bond, , we've, been, waiti..."
2,4175,"[""a, heart-wrenching, and, uplifting, story, o..."
3,9448,"[a, fast-paced, and, engrossing, story, a, def..."
4,14535,"[gayle, jackson, sloan, is, a, native, of, phi..."


In [385]:
for v in vocab:
    combined_words[v] = combined_words['all_text_parsed'].apply(lambda x: x.count(" " + v + " "))

/home/unixinternet/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [252]:
combined_words

,product_id,all_text_parsed,,Paperback,!,"""","""A","""I","""The","""This",...,"you,",you.,young,younger,youngest,your,yourself,"yourself,",yourself.,youth
0,12572,"""Fairy phobic or trapped in a true-life fairy ...",33,0,0,0,0,0,0,0,...,0,1,0,0,0,2,0,0,0,0
1,4397,"Welcome back, Mr Bond. We've been waiting for...",33,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,4175,"""A heart-wrenching and uplifting story of surv...",4,0,0,0,0,0,0,0,...,0,0,1,0,0,2,0,0,0,0
3,9448,A fast-paced and engrossing story. A definite ...,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14535,Gayle Jackson Sloan is a native of Philadelphi...,15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1915,Keri Smith a freelance illustrator living in C...,19,0,0,0,0,0,0,0,...,0,0,0,1,0,5,1,0,0,0
6,6919,Peter G. Tsouras is noted military historian a...,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,4107,"""...adults version of Dr. Seusss Oh, the Place...",15,0,0,0,0,0,0,0,...,1,0,1,0,0,9,0,0,0,0
8,4199,Dr. Goldberg received his hypnosis training in...,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2325,"""Von Franz has a remarkable gift for interpret...",3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [253]:
# The easiest way is to pickle it using to_pickle:
combined_words.to_pickle("vocab_matrix")  # where to save it, usually as a .pkl
# Then you can load it back using:
# df = pd.read_pickle(file_name)

## convert to tfidf

In [267]:
from sklearn.feature_extraction.text import TfidfTransformer
tf = TfidfTransformer()
tfidf_words = tf.fit_transform(combined_words[vocab])
tfidf_words

<6568x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 1748528 stored elements in Compressed Sparse Row format>

In [278]:
tfidf_df = pd.SparseDataFrame(tfidf_words)
tfidf_df.columns = vocab
tfidf_df["product_id"] = combined_words["product_id"]
tfidf_df = tfidf_df[["product_id"] + vocab]
tfidf_df.head()

,product_id,,Paperback,!,"""","""A","""I","""The","""This","""the",...,"you,",you.,young,younger,youngest,your,yourself,"yourself,",yourself.,youth
0,12572,0.272441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.02301,NaN,NaN,NaN,0.026226,NaN,NaN,NaN,NaN
1,4397,0.301791,0.035262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.031847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4175,0.072792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.050358,NaN,NaN,0.057810,NaN,NaN,NaN,NaN
3,9448,0.054629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14535,0.188670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## get training data for a single user 

for a user, find all the movies they rated.
create a table
product_id, rating, \[vocab columns\]

find the n words with the highest combined tf-idfs over all the rows 
use these words as the features
run logistic regression

In [286]:
user_ratings = train['user_product_ratings']
combined_tfidf = user_ratings.merge(tfidf_df, on='product_id')
combined_tfidf.loc[user_ratings['user_id'] == 508]

,product_id,rating_x,user_id,,Paperback,!,"""","""A","""I","""The",...,"you,",you.,young,younger,youngest,your,yourself,"yourself,",yourself.,youth
0,6155,5.0,508,0.242873,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.041244,NaN,0.064295,0.031739,NaN,NaN,NaN
51,8247,5.0,508,0.152980,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.060475,NaN,NaN,0.017356,NaN,NaN,NaN,NaN
1364,15517,5.0,508,0.179761,NaN,NaN,NaN,NaN,0.007949,NaN,...,NaN,NaN,0.087512,0.013567,0.008975,0.018506,NaN,NaN,NaN,NaN
1799,6547,5.0,508,0.153725,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.038672,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,15692,5.0,508,0.234177,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.018598,NaN,NaN,NaN,NaN
1807,9684,4.0,508,0.030292,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.041912,0.061729,NaN,0.048115,NaN,NaN,NaN,NaN
1808,1737,4.0,508,0.099432,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.055030,NaN,NaN,0.055277,0.015593,NaN,NaN,NaN
4311,15826,5.0,508,0.246889,NaN,NaN,NaN,NaN,NaN,0.038057,...,NaN,NaN,0.016663,NaN,NaN,0.038259,NaN,NaN,NaN,NaN
4615,8797,5.0,508,0.215257,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.070078,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4621,2053,5.0,508,0.168541,NaN,0.003529,NaN,NaN,NaN,NaN,...,0.002743,NaN,0.028332,0.003210,0.004247,0.018764,NaN,NaN,NaN,NaN


In [333]:
def extract_user_vocab(user):
    ### Extract 200 words with the most significance among all descriptions/reviews of the products that the user has rated
    user_product_tfidf = combined_tfidf.loc[user_ratings['user_id'] == user]
    tfidf_cum = combined_tfidf.mean(axis=0)[4:]
    personalized_vocab = tfidf_cum.sort_values(ascending=False)[0:200].keys().tolist()
    return personalized_vocab

In [344]:
def feature_extractor(product, personalized_vocab):
#     movie = 12572
    text = list(combined.loc[combined['product_id'] == product]['all_text'])[0]
    print(text)
    features = [ text.count(v) for v in personalized_vocab]
    return features

In [345]:
# def get_all_training_data_for_user(user):
user = 508
user_vocab = extract_user_vocab(user)
products_ratings = user_ratings.loc[user_ratings['user_id'] == user] # = train['user_product_ratings']
products = list(products_ratings["product_id"])
# print(products)
ratings = list(products_ratings["rating"])
X = np.array([feature_extractor(p, user_vocab) for p in products])
print(np.max(X))
# print(ratings)

['By Stan Berenstain and Jan Berenstain, Illustrated by Stan Berenstain and Jan Berenstain', "for getting your kid introduced to his/her ABC's. I also have the Berenstain Bear's A Book and C Book. My kids love the funny pictures and topical sentences.", 'This Book is funny and is full of B words, like "big brown bear, blue, bull, beautiful baboon...."  Another B phrase is added each time so by the end it\'s a long phrase.  There is a surprise at the end for a sad Baby Bird.  (my daughter dictated that paragraph)This isn\'t Berenstain Bear family book, with Mama, Papa, Sister and Brother but is a celebration of an important letter.  It\'s fun to read out loud with a swingy feel to it.', 'A favorite Berenstain book of my children I was happy to find it for my grandson.', 'This book is quite funny.  Especially when you read it fast as I\'m always requested to do.  How fast can you read, "big brown bear, blue babboon, beautiful babboon blowing bubbles, biking backward, bump black bug\'s ba

In [341]:
print(user_vocab)

['Dilbert', 'Chomsky', 'Sharpe', 'Pimsleur', 'Abby', 'MacKade', 'Rae', 'the', 'Cassie', 'Voyager', 'flaps', 'SQL', 'PHP', 'deck', 'Photoshop', 'Tucker', 'sock', 'Hobbes', 'Perl', 'Buffy', 'Lily', 'manga', 'Java', 'Edie', 'yoga', 'Calvin', 'Carla', 'Linux', 'Briggs', 'chess', 'knitting', 'and', 'Ashley', 'Superman', 'quilt', 'Drake', 'MacArthur', 'Tracie', 'Amy', 'Rule', 'poems', 'Batman', 'watercolor', 'Orthodox', 'recipes', 'Sister', 'Heitzmann', 'Jenny', 'Spanish', 'Doreen', 'of', 'puzzles', 'cards', 'Brad', 'Emma', 'Brenda', 'Wright', 'Islam', 'Yoga', 'Lauren', 'Lori', 'a', 'Osprey', 'guitar', 'Kristen', 'Joel', 'to', 'Lindsey', 'Roberts', 'crochet', 'Davis', 'Louise', 'Mercy', 'Francis', 'Microsoft', 'Carol', 'martial', 'Islamic', 'Beverly', 'Forgotten', 'Visual', 'Lynn', 'Ruby', 'Loretta', 'Jamie', 'Mercedes', 'Lance', 'Windows', 'Nora', 'Claire', 'Pope', 'Japanese', 'Trek', 'Chase', 'Dana', 'stitch', 'Foster', 'plants', 'Stanley', 'Warren', 'Williams', 'Tanya', 'crystal', 'Cathol